In [1]:
import os
import sys
sys.path.append('path/to/dir')

In [2]:
from Sample import Sample
import matplotlib.pyplot as plt
from segmentation.segmentation import Segmentation
from skimage.segmentation import find_boundaries
import cv2 
import numpy as np
import preprocessing.preprocess as pp
import registration.register as reg

In [3]:
# Load the sample from the respective directory
sample_directory = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_tmp_simon/20211214_18-2600_BM"
save_directory = "/save/dir"
example = Sample(sample_directory)

/home/simon_g/workflow/Sample.py:94: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  self.if_imgs = np.array([cv2.cvtColor(cv2.imread(df[x]), cv2.COLOR_BGR2GRAY) for x in ["if_b", "if_g", "if_r"]])


In [4]:
#fig, ax = plt.subplots(5, len(example.rois), figsize=(15, 10))

for i, roi in enumerate(example.rois):

    pp_if = pp.preprocess(roi.if_nuc)
    pp_imc = pp.preprocess(roi.imc_nuc_upscaled)

    #ax[0, i].imshow(roi.if_nuc)
    #ax[1, i].imshow(pp_if)
    #ax[2, i].imshow(roi.imc_nuc)
    #ax[3, i].imshow(pp_imc)

    h = reg.find_matches(pp_if, pp_imc)
    transformed = reg.transform(pp_if, pp_imc, h)
    #ax[4, i].imshow(additive_blend(transformed, pp_imc))
    example.rois[i].if_registered = transformed
    print(f"Registered {roi.roi_num}")
    #plt.tight_layout()
#plt.show()


Found 1456 matches.
Found 484 matches.
Found 1785 matches.


In [ ]:
# Initialize the Segmentation Class with thre correct resolution
Mesmer = Segmentation("mesmer", mpp=0.175)

In [ ]:
# Segment the IF DAPI Image
masks = Mesmer(roi.if_nuc)

In [ ]:
# lets look at the masks
rgb_data = cv2.cvtColor(roi.if_nuc, cv2.COLOR_GRAY2RGB)
boundaries = np.zeros_like(roi.if_nuc)
overlay_data = np.copy(rgb_data)

boundary = find_boundaries(masks, connectivity=0, mode='outer')
boundaries[boundary > 0] = 1

overlay_data[boundaries > 0] = (255,0,0)

plt.rcParams["figure.figsize"] = (20, 20)
plt.imshow(overlay_data)


In [ ]:
# hier noch schauen mit dem Registration, dass der Output hier nicht 700 x 700 sondern 4096 auf 4096 ist, einfach vor der registrierung das IMC bild upskalieren und dann darafu registrieren aber nicht speichern.


#fig, ax = plt.subplots(5, len(example.rois), figsize=(15, 10))

for i, roi in enumerate(example.rois):

    pp_if = pp.preprocess(roi.if_nuc)
    pp_imc = pp.preprocess(roi.imc_nuc_upscaled)

    #ax[0, i].imshow(roi.if_nuc)
    #ax[1, i].imshow(pp_if)
    #ax[2, i].imshow(roi.imc_nuc)
    #ax[3, i].imshow(pp_imc)

    h = reg.find_matches(pp_if, pp_imc)
    transformed = reg.transform(pp_if, pp_imc, h)
    #ax[4, i].imshow(additive_blend(transformed, pp_imc))

    #plt.tight_layout()
#plt.show()